In [1]:
import time, random, re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from tenacity import retry, stop_after_attempt, wait_exponential

In [2]:
# إعدادات عامة

BASE_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/126.0 Safari/537.36"
    )
}

# صفحات البحث (أول صفحة لكل تصنيف)
# ملاحظة: أقدر أعدل القيم أو أضيف تصنيفات جديدة لاحقاً
CATEGORIES = {
    "IT": "https://www.wadhefa.com/jobfind.php?action=search&bx_jtitle=&rdjt=2&jids%5B%5D=75&jids%5B%5D=76&jids%5B%5D=84&lids%5B%5D=000&bx_prv%5B%5D=&bx_plng%5B%5D=0&bx_kwd=&rdKeyw=2&bx_minsalary=&bx_maxsalary=&bx_lngids%5B%5D=-&rdLang=2&tids%5B%5D=0&posted=0&o=1&o_show=3&cmdSearch=++%D8%A8%D8%AD%D8%AB++",
    "Marketing": "https://www.wadhefa.com/jobfind.php?action=search&bx_jtitle=&rdjt=2&jids%5B%5D=72&jids%5B%5D=73&jids%5B%5D=74&lids%5B%5D=000&bx_prv%5B%5D=&bx_plng%5B%5D=0&bx_kwd=&rdKeyw=2&bx_minsalary=&bx_maxsalary=&bx_lngids%5B%5D=-&rdLang=2&tids%5B%5D=0&posted=0&o=1&o_show=3&cmdSearch=++%D8%A8%D8%AD%D8%AB++",
    "Finance": "https://www.wadhefa.com/jobfind.php?action=search&bx_jtitle=&rdjt=2&jids%5B%5D=64&jids%5B%5D=77&jids%5B%5D=79&lids%5B%5D=000&bx_prv%5B%5D=&bx_plng%5B%5D=0&bx_kwd=&rdKeyw=2&bx_minsalary=&bx_maxsalary=&bx_lngids%5B%5D=-&rdLang=2&tids%5B%5D=0&posted=0&o=1&o_show=3&cmdSearch=++%D8%A8%D8%AD%D8%AB++",
    "Engineering": "https://www.wadhefa.com/jobfind.php?action=search&bx_jtitle=&rdjt=2&jids%5B%5D=80&jids%5B%5D=81&jids%5B%5D=82&jids%5B%5D=83&jids%5B%5D=84&jids%5B%5D=85&lids%5B%5D=000&bx_prv%5B%5D=&bx_plng%5B%5D=0&bx_kwd=&rdKeyw=2&bx_minsalary=&bx_maxsalary=&bx_lngids%5B%5D=-&rdLang=2&tids%5B%5D=0&posted=0&o=1&o_show=3&cmdSearch=++%D8%A8%D8%AD%D8%AB++",
    "Healthcare": "https://www.wadhefa.com/jobfind.php?action=search&bx_jtitle=&rdjt=2&jids%5B%5D=96&jids%5B%5D=97&jids%5B%5D=98&jids%5B%5D=99&lids%5B%5D=000&bx_prv%5B%5D=&bx_plng%5B%5D=0&bx_kwd=&rdKeyw=2&bx_minsalary=&bx_maxsalary=&bx_lngids%5B%5D=-&rdLang=2&tids%5B%5D=0&posted=0&o=1&o_show=3&cmdSearch=++%D8%A8%D8%AD%D8%AB++",
    "Education": "https://www.wadhefa.com/jobfind.php?action=search&bx_jtitle=&rdjt=2&jids%5B%5D=67&lids%5B%5D=000&bx_prv%5B%5D=&bx_plng%5B%5D=0&bx_kwd=&rdKeyw=2&bx_minsalary=&bx_maxsalary=&bx_lngids%5B%5D=-&rdLang=2&tids%5B%5D=0&posted=0&o=1&o_show=3&cmdSearch=++%D8%A8%D8%AD%D8%AB++",
}

# عدد الروابط المراد سحبها من الصفحة الأولى لكل تصنيف
LINKS_PER_CATEGORY = 100  # أقدر أرفعها إلى 300

In [3]:
SELECTORS = {
    "listing_link": 'a.tablelist[href*="/details/job/"]',  # روابط الإعلانات في صفحة القوائم
    "next_link_candidates": 'a[rel="next"], a:contains("التالي"), a:contains("الصفحة التالية")'
}

DETAIL_SELECTORS = {
    "title": "td.view",           # عنوان الإعلان
    "description": "td.td4textarea"  # نص الوصف/المتطلبات
}

In [4]:
# دوال مساعدة

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=1, max=8))
def fetch(url: str) -> str:
    """جلب صفحة HTML مع إعادة محاولة تلقائية عند الفشل المؤقت."""
    resp = requests.get(url, headers=BASE_HEADERS, timeout=25)
    resp.raise_for_status()
    return resp.text

def extract_links_from_list_page(list_url: str, limit: int = 300):
    """استخراج أول N روابط إعلانات من صفحة القوائم (بدون تنقل صفحات)."""
    html = fetch(list_url)
    soup = BeautifulSoup(html, "lxml")
    links = []
    for a in soup.select('a.tablelist[href*="/details/job/"]'):
        href = a.get("href")
        if href:
            links.append(urljoin(list_url, href))
        if len(links) >= limit:
            break
    # إزالة التكرارات مع الحفاظ على الترتيب
    return list(dict.fromkeys(links))

AR_DIACRITICS_RE = re.compile(r"[\u064B-\u0652]")

def normalize_ar_keep_ascii(text: str) -> str:
    """تنظيف خفيف للنص العربي مع الإبقاء على ASCII للإيميلات/الأرقام."""
    if not text:
        return ""
    t = text.replace("\u00A0"," ")
    t = re.sub(r"\u0640", "", t)                 # إزالة التطويل
    t = AR_DIACRITICS_RE.sub("", t)              # إزالة التشكيل
    t = t.replace("أ","ا").replace("إ","ا").replace("آ","ا") \
         .replace("ى","ي").replace("ؤ","و").replace("ئ","ي")
    t = re.sub(r"[ \t]+", " ", t).strip()
    return t

def excel_safe(s: str) -> str:
    """منع Excel من تفسير النص كمعادلة إذا بدأ بـ = أو + أو - أو @ أو Tab."""
    if not s:
        return ""
    if s[:1] in ("=", "+", "-", "@", "\t"):
        return "'" + s
    return s

In [5]:
# استخراج تفاصيل إعلان واحد (العنوان + الوصف)
# ملاحظة مهمة: عنوان الوظيفة داخل td[width="30%"] a.tablelist
# والوصف داخل td.td4textarea

def parse_job_detail(job_url: str, category_label: str):
    html = fetch(job_url)
    soup = BeautifulSoup(html, "lxml")

    # العنوان
    t_el = soup.select_one('td[width="30%"] a.tablelist')
    title_raw = t_el.get_text(" ", strip=True) if t_el else ""
    title = normalize_ar_keep_ascii(title_raw)

    # الوصف
    d_el = soup.select_one("td.td4textarea")
    desc_raw = d_el.get_text("\n", strip=True) if d_el else ""
    description = normalize_ar_keep_ascii(desc_raw)

    # حماية Excel
    return {
        "category": category_label,
        "title": excel_safe(title),
        "description": excel_safe(description),
    }

In [6]:
# سحب البيانات لكل التصنيفات وتجميعها في DataFrame واحد

all_rows = []
for label, list_url in CATEGORIES.items():
    links = extract_links_from_list_page(list_url, limit=LINKS_PER_CATEGORY)
    print(f"{label}: استخرجت {len(links)} رابط من الصفحة الأولى")
    for u in links:
        try:
            row = parse_job_detail(u, category_label=label)
            all_rows.append(row)
            # تأخير لطيف لتجنب ضغط الموقع
            time.sleep(random.uniform(0.6, 1.2))
        except Exception as e:
            print("فشل:", u, e)

df = pd.DataFrame(all_rows).drop_duplicates()
print("إجمالي السجلات:", len(df))
df.head(5)

IT: استخرجت 100 رابط من الصفحة الأولى
Marketing: استخرجت 100 رابط من الصفحة الأولى
Finance: استخرجت 100 رابط من الصفحة الأولى
Engineering: استخرجت 100 رابط من الصفحة الأولى
Healthcare: استخرجت 100 رابط من الصفحة الأولى
Education: استخرجت 100 رابط من الصفحة الأولى
إجمالي السجلات: 577


,category,title,description
0,IT,,تعلن شركة تقنية معلومات عن توفر شاغر وظيفي بمس...
1,IT,,"'- تصميم الشبكات (LAN, WAN, WLAN)\n- ادارة اجه..."
2,IT,,'- مطلوب رسام اوتوكاد للرسم وقراءة المخططات وح...
3,IT,,الوصف الوظيفي:\n- تطوير وبناء تطبيقات موبايل م...
4,IT,,'- مطلوب مبرمج لديه خبرة في في Oracle APEX للع...


In [7]:
# إحصائية سريعة: عدد السجلات لكل تصنيف (لضمان التوازن النسبي)
counts = df["category"].value_counts()
print(counts)

category
Marketing      99
IT             98
Finance        97
Engineering    95
Education      95
Healthcare     93
Name: count, dtype: int64


In [8]:
# الحفظ إلى CSV (ثلاث أعمدة فقط)
# ملاحظة: لو الملف مفتوح في Excel راح يطلع PermissionError — غير الاسم أو أغلق Excel

out_path = "wadhefa_dataset.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"تم الحفظ في {out_path}")

تم الحفظ في wadhefa_dataset.csv


In [9]:
# ### (اختياري) تصحيح CSV قديم لو واجهت مشكلة #NAME? في Excel
# هذا البلوك يصلّح عمود الوصف والعنوان بإضافة apostrophe لو بدأت بـ = أو + أو - أو @

def fix_csv_for_excel(input_csv: str, output_csv: str):
    tmp = pd.read_csv(input_csv, encoding="utf-8-sig")
    for col in ["title", "description"]:
        if col in tmp.columns:
            tmp[col] = tmp[col].map(excel_safe)
    tmp.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"تم إنشاء ملف آمن: {output_csv}")